# AO3 query search

A notebook to use scrape metadata from the fanwork archive AO3 and store it in mySQL files. 

In [1]:
# import specialised functions from ao3.py file
%load_ext autoreload
%autoreload 1
%aimport ao3

In [2]:
# necessary libraries. One day I will use beautiful soup.
import requests
import os
import time
import datetime

from bs4 import BeautifulSoup
import string

import math
import sqlite3
import pandas as pd
import numpy as np
import ast


Initiate the dataFrame for query 

In [3]:
url = 'https://archiveofourown.org/tags/Ronnie%20Lee/works'
#url = 'https://archiveofourown.org/works?utf8=%E2%9C%93&work_search%5Bsort_column%5D=revised_at&work_search%5Bother_tag_names%5D=&work_search%5Bexcluded_tag_names%5D=&work_search%5Bcrossover%5D=&work_search%5Bcomplete%5D=&work_search%5Bwords_from%5D=&work_search%5Bwords_to%5D=&work_search%5Bdate_from%5D=&work_search%5Bdate_to%5D=&work_search%5Bquery%5D=&work_search%5Blanguage_id%5D=zh&commit=Sort+and+Filter&tag_id=Miraculous+Ladybug'
dbName = 'test_ronnie'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
root_url = 'https://archiveofourown.org'

# Start the database
df = pd.DataFrame()
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
works = soup.find_all(role="article")
for fic in works:
    ent = ao3.query_AO3_work(fic)
    df=df.append(ent,ignore_index=True)
    
u = soup.find_all(rel="next")
if len(u)>0:
    v = BeautifulSoup(str(u[0]),'html.parser')
    NEXTurl = root_url + str(v.a.get('href'))
    print(NEXTurl)
    NextPage = True

ficCount = len(df)
df.loc[ficCount-5:]

https://archiveofourown.org/tags/Ronnie%20Lee/works?page=2


,Bookmarks,Category,Chapters,Character_tags,Collection,Comments,Complete,Creator,Date,Fandom,...,Series,Series_part,Summary,Tags,Title,Warning,Words,cURL,sURL,url
15,1.0,"[Gen, F/F]","[6, 6]","[Ronnie Lee, Ray Butani, Patrick Brewer, The F...",1.0,43.0,Complete Work,"[Amanita_Fierce, DoubleL27]",20201213.0,[Schitt's Creek],...,[],[],"\n“Hello, Ronnie!” Ray yelled, slipping out of...","[Ray Butani & Ronnie Lee, Patrick Brewer & Ron...",Come In From The Cold,[No Archive Warnings Apply],14977.0,[https://archiveofourown.org/users/Amanita_Fie...,[],https://archiveofourown.org/works/27769942
16,16.0,[M/M],"[8, 8]","[David Rose (Schitt's Creek), Patrick Brewer, ...",1.0,29.0,Complete Work,[perfect-porcelain (tedddylupin)],20201212.0,[Schitt's Creek],...,[],[],David Rose is a washed up childhood actor who ...,"[Patrick Brewer/David Rose, David Rose (Schitt...",the christmas chateau,[No Archive Warnings Apply],19092.0,[https://archiveofourown.org/users/tedddylupin...,[],https://archiveofourown.org/works/27750664
17,14.0,[No category],"[5, 5]","[Patrick Brewer, David Rose (Schitt's Creek), ...",0.0,32.0,Complete Work,[writemenot],20201210.0,[Schitt's Creek],...,[],[],"\n“I’m really glad you stole my boss’s dinner,...","[Patrick Brewer/David Rose, Patrick Brewer, Da...",you got my future (in your hands),[Choose Not To Use Archive Warnings],26578.0,[https://archiveofourown.org/users/writemenot/...,[],https://archiveofourown.org/works/27570559
18,37.0,[M/M],"[1, 1]","[David Rose (Schitt's Creek), Patrick Brewer, ...",1.0,82.0,Complete Work,[this_is_not_nothing],20201210.0,[Schitt's Creek],...,[],[],"For the prompt: ""Who doesn’t love a wintry or ...","[Patrick Brewer/David Rose, David Rose (Schitt...",How Sweet It Is,[No Archive Warnings Apply],11802.0,[https://archiveofourown.org/users/this_is_not...,[],https://archiveofourown.org/works/27758566
19,8.0,[M/M],"[1, 1]","[Patrick Brewer, David Rose (Schitt's Creek), ...",1.0,64.0,Complete Work,[mycleverusername],20201208.0,[Schitt's Creek],...,[],[],"\n“David, I’ll have you know that I take gift ...","[Patrick Brewer/David Rose, Patrick Brewer, Da...",present company excepted,[No Archive Warnings Apply],3550.0,[https://archiveofourown.org/users/mycleveruse...,[],https://archiveofourown.org/works/27765349


Continue to scrape subsequent pages of query into database (pandas dataframe.)

In [4]:
while NextPage:
    time.sleep(5)
    page = requests.get(NEXTurl, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    works = soup.find_all(role="article")
    for fic in works:
        ent = ao3.query_AO3_work(fic)
        df=df.append(ent,ignore_index=True)

    u = soup.find_all(rel="next")
    if len(u)>0:
        v = BeautifulSoup(str(u[0]),'html.parser')
        NEXTurl = root_url + str(v.a.get('href'))
        print(NEXTurl)
        NextPage = True
    else:
        NextPage = False

ficCount = len(df)
df.loc[ficCount-3:]

https://archiveofourown.org/tags/Ronnie%20Lee/works?page=3
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=4
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=5
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=6
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=7
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=8
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=9
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=10
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=11
https://archiveofourown.org/tags/Ronnie%20Lee/works?page=12


,Bookmarks,Category,Chapters,Character_tags,Collection,Comments,Complete,Creator,Date,Fandom,...,Series,Series_part,Summary,Tags,Title,Warning,Words,cURL,sURL,url
227,54.0,[M/M],"[1, 1]","[David Rose (Schitt's Creek), Patrick Brewer, ...",0.0,13.0,Complete Work,[mihaly],20181207.0,[Schitt's Creek],...,[],[],Patrick wants to move their relationship forwa...,"[Patrick Brewer/David Rose, David Rose (Schitt...",no need to worry,[No Archive Warnings Apply],4000.0,[https://archiveofourown.org/users/mihaly/pseu...,[],https://archiveofourown.org/works/16895190
228,143.0,[M/M],"[1, 1]","[David Rose, Patrick Brewer, Stevie Budd, Moir...",0.0,57.0,Complete Work,[falconeggs],20181129.0,[Schitt's Creek],...,[],[],“Patrick loves David the exact same way Johnny...,"[Patrick Brewer/David Rose, Minor Johnny Rose/...",Special Order Forms,[No Archive Warnings Apply],10164.0,[https://archiveofourown.org/users/falconeggs/...,[],https://archiveofourown.org/works/16782049
229,4.0,"[F/M, M/M]","[5, 5]","[Johnny Rose, Moira Rose, Original Male Charac...",0.0,9.0,Complete Work,[Miso],20180308.0,[Schitt's Creek],...,[],[],Johnny's life is going fine. Until a certain s...,"[Johnny Rose/Moira Rose, Johnny Rose/Original ...",Without A Paddle,[No Archive Warnings Apply],3561.0,[https://archiveofourown.org/users/Miso/pseuds...,[],https://archiveofourown.org/works/13910523


Comb through database and drop any duplicates

In [5]:
print(len(df))
df = df.drop_duplicates(subset=['Node'])

print(len(df))

230
230


Save scaped data as a table in the AO3 mySQL database.

Note: because of the limited data types of mySQL (relative to pandas), the feilds containing lists must be converted strings, and panda's default numpy.floats to integers. 

In [7]:
os.chdir('/Users/finn/dev/Ao3mine/data')
TODAY = datetime.date.today().isoformat()

dbName = 'test_'+ dbName + '_' + str(TODAY)
conn = sqlite3.connect(dbName + '_ao3DB.db')  
c = conn.cursor()
c.execute('''CREATE TABLE WORKS
             ([Bookmarks] integer, [Category] text, [Chapters] text, [Character_tags] text, [Collection] text, [Comments] integer, [Complete] text, [Creator] text, [Date] integer, [Fandom] text, [Freeform_tags] text, [Gift] text, [Hits] integer, [Kudos] integer, [Language] text, [Node] integer, [Rating] text, [Relationship_tags] text, [Series] text, [Series_part] text, [Summary] text, [Tags] text, [Title] text, [Warning] text, [Words] integer, [cURL] text, [sURL] text, [url] text)''')

df_sql = ao3.db2sqlTypes(df)
df_sql.to_sql('WORKS', conn, if_exists='append', index = False) # Insert the values from the csv file into the table 'CLIENTS' 


In [10]:
# in case you need to repeat an evaluation within a day
# c.execute('''DROP TABLE WORKS''')


Now an example of how to extract a database from evaluating in dataframe form.

In [8]:
c.execute("SELECT * FROM WORKS")
data = c.fetchall()
cols = [tuple[0] for tuple in c.description]
df_full = ao3.sql2dbListTypes(data,cols)
df_full

,Bookmarks,Category,Chapters,Character_tags,Collection,Comments,Complete,Creator,Date,Fandom,...,Series,Series_part,Summary,Tags,Title,Warning,Words,cURL,sURL,url
0,35,[M/M],"[9, 9]","[David Rose (Schitt's Creek), Patrick Brewer, ...",0,133,Complete Work,[Lisamc21],20201226,[Schitt's Creek],...,[],[],Patrick and David are coworkers at the headqua...,"[Patrick Brewer/David Rose, David Rose (Schitt...",Secret Santa is a Scam,[No Archive Warnings Apply],37201,[https://archiveofourown.org/users/Lisamc21/ps...,[],https://archiveofourown.org/works/28159230
1,4,[M/M],"[1, 1]","[David Rose (Schitt's Creek), Patrick Brewer, ...",0,54,Complete Work,[JustWaiting23],20201226,[Schitt's Creek],...,[The Cat Chronicles],[3],\n“Don’t you think that’s just awful? That he ...,"[Patrick Brewer/David Rose, Ronnie Lee & David...",The Rumours are Terrible,[No Archive Warnings Apply],3513,[https://archiveofourown.org/users/JustWaiting...,[https://archiveofourown.org/series/2065569],https://archiveofourown.org/works/28342500
2,3,[Multi],"[44, ?]","[Moira Rose, Johnny Rose, Alexis Rose, David R...",0,171,Work in Progress,[mrenae],20201226,[Schitt's Creek],...,[],[],What if Moira was the one who took the pregnan...,"[Johnny Rose/Moira Rose, Alexis Rose & David R...",Miracles Don’t Take Reservations,[Choose Not To Use Archive Warnings],14254,[https://archiveofourown.org/users/mrenae/pseu...,[],https://archiveofourown.org/works/27820783
3,75,[M/M],"[37, ?]","[David Rose (Schitt's Creek), Patrick Brewer, ...",0,944,Work in Progress,[PandorasDaydream],20201225,[Schitt's Creek],...,[],[],This series (Part 4) starts before 5.01 and wi...,"[Patrick Brewer/David Rose, David Rose (Schitt...",Language of Love: Part 4 (Season 5),[Choose Not To Use Archive Warnings],338839,[https://archiveofourown.org/users/PandorasDay...,[],https://archiveofourown.org/works/22219387
4,7,[M/M],"[2, 2]","[David Rose, Patrick Brewer, Alexis Rose, John...",0,22,Complete Work,[fairmanor],20201224,[Schitt's Creek],...,[],[],"New York, 1895, and David Rose is at his wit’s...","[Patrick Brewer/David Rose, David Rose, Patric...",Saving Grace,[No Archive Warnings Apply],27225,[https://archiveofourown.org/users/fairmanor/p...,[],https://archiveofourown.org/works/27581615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,14,[M/M],"[7, 7]","[Patrick Brewer, David Rose, Ronnie (Schitt's ...",0,28,Complete Work,[yerbamansa],20190114,[Schitt's Creek],...,[],[],Patrick has been keeping a secret from David: ...,"[Patrick Brewer/David Rose, Patrick Brewer, Da...",It’s where I want to be,[No Archive Warnings Apply],11242,[https://archiveofourown.org/users/yerbamansa/...,[],https://archiveofourown.org/works/17307665
226,23,[F/F],"[1, 1]","[Alexis Rose, Twyla Sands, Ronnie (Schitt's Cr...",1,16,Complete Work,[in48frames],20181218,[Schitt's Creek],...,[],[],"“Oh,” Twyla says. “Yeah. Schitt’s Creek is sup...","[Alexis Rose/Twyla Sands, Alexis Rose, Twyla S...","Oh Please, Not Now",[No Archive Warnings Apply],12197,[https://archiveofourown.org/users/in48frames/...,[],https://archiveofourown.org/works/17046239
227,54,[M/M],"[1, 1]","[David Rose (Schitt's Creek), Patrick Brewer, ...",0,13,Complete Work,[mihaly],20181207,[Schitt's Creek],...,[],[],Patrick wants to move their relationship forwa...,"[Patrick Brewer/David Rose, David Rose (Schitt...",no need to worry,[No Archive Warnings Apply],4000,[https://archiveofourown.org/users/mihaly/pseu...,[],https://archiveofourown.org/works/16895190
228,143,[M/M],"[1, 1]","[David Rose, Patrick Brewer, Stevie Budd, Moir...",0,57,Complete Work,[falconeggs],20181129,[Schitt's Creek],...,[],[],“Patrick loves David the exact same way Johnny...,"[Patrick Brewer/David Rose, Minor Johnny Rose/...",Special Order Forms,[No Archive Warnings Apply],10164,[https://archiveofourown.org/users/falconeggs/...,[],https://archiveofourown.org/works/16782049


In [9]:
# test outputs
df_full.loc[3,'Tags'][1]

"David Rose (Schitt's Creek)"